In [2]:
import os
os.chdir('../')

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor

houses = pd.read_csv('Datasets/houses.csv')
houses.head()

X = houses.drop(['id','price','yr_built','yr_renovated'], axis = 1)
y = houses.price

y_binned = pd.qcut(y, q=20)
X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    test_size=0.3, 
                                                    random_state=617,
                                                    stratify=y_binned)



In [19]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import VotingRegressor

reg = LinearRegression()
reg.fit(X_train, y_train)

tree = DecisionTreeRegressor(random_state=617)
tree.fit(X_train, y_train)

DecisionTreeRegressor(random_state=617)

In [20]:
models = [('Linear Regression',reg),
              ('Regression Tree',tree)]

for model_name, model in models:
    model.fit(X_train, y_train)
    pred_test = model.predict(X_test)
    rmse = root_mean_squared_error(y_test, pred_test)
    print(model_name, rmse)

Linear Regression 218930.99331410075
Regression Tree 269138.06694233755


In [ ]:
from sklearn.ensemble import VotingRegressor

vc=VotingRegressor([('Tree',tree),('regression',reg)],n_jobs=-1)
vc.fit(X_train,y_train)
pred_train=vc.predict(X_train)
pred_test=vc.predict(X_test)

In [28]:
root_mean_squared_error(y_train,pred_train),root_mean_squared_error(y_test,pred_test)

(107829.28273583231, 219044.94791252245)

In [29]:
%%time
from sklearn.ensemble import BaggingRegressor
bag = BaggingRegressor(n_estimators=200,
                        n_jobs = -1, 
                        oob_score=True,
                        random_state=1031) # default estimator is DecisionTreeRegressor
bag.fit(X_train, y_train)

CPU times: total: 1.84 s
Wall time: 8.91 s


BaggingRegressor(n_estimators=200, n_jobs=-1, oob_score=True, random_state=1031)

In [30]:
pred_train = bag.predict(X_train)
pred_test = bag.predict(X_test)

print('Train RMSE: ', root_mean_squared_error(y_train, pred_train))
print('Test RMSE:  ', root_mean_squared_error(y_test, pred_test))

Train RMSE:  71330.1883538567
Test RMSE:   194798.81646852565
